In [2]:
# importing packages and libraries
from mxnet import nd, gluon, autograd
from mxnet.gluon import nn
import mxnet as mx
import numpy as np
import pandas as pd
import time
from mxnet import profiler
import re

In [3]:
data_ctx = mx.gpu()
model_ctx = mx.gpu()


In [4]:
net = nn.Sequential()
net.add(
    # Here, we use a larger 11 x 11 window to capture objects. At the same time,
   # we use a stride of 4 to greatly reduce the height and width of the output.
    nn.Conv2D(96,kernel_size=11,strides=4,activation='relu'),
    nn.MaxPool2D(pool_size=3,strides=2),
    # Make the convolution window smaller, set padding to 2 for consistent
    # height and width across the input and output, and increase the
    # number of output channels
    nn.Conv2D(256,kernel_size=5,padding=2,activation='relu'),
    nn.MaxPool2D(pool_size=3,strides=2),
    # Use three successive convolutional layers and a smaller convolution
    # window. Except for the final convolutional layer, the number of
    # output channels is further increased. Pooling layers are not used to
    # reduce the height and width of input after the first two
    # convolutional layers
   nn.Conv2D(384, kernel_size=3,padding=1,activation='relu'),
   nn.Conv2D(384, kernel_size=3,padding=1,activation='relu'),
   nn.Conv2D(256, kernel_size=3,padding=1,activation='relu'),
   nn.MaxPool2D(pool_size=3, strides=2),
   nn.Dense(4096,activation='relu'),
   nn.Dropout(.5),
   nn.Dense(4096,activation='relu'),
   nn.Dropout(0.5),
   nn.Dense(10)

)

In [5]:
X = np.random.uniform(size=(1,1,224,224))
net.initialize()
for layer in net:
  X=layer(X)
  print(layer.name,'output shape:\t',X.shape)

AssertionError: HybridBlock requires the first argument to forward be either Symbol or NDArray, but got <class 'numpy.ndarray'>